In [20]:
from random import randint

class Person:

    def __init__(self, name, age, wallet):
        self.name = name
        self.age = age
        self.wallet = wallet
        print(f"A person called {name} is created.\n")

In [21]:
class Card:

    def __init__(self, suit, rank, lost = False):
        self.suit = suit
        self.rank = rank
        
        '''
        print(f"created card: {self.rank} of {self.suit}")
        
        '''
        
    def __str__(self):
        return f"{self.rank} of {self.suit}"

In [22]:
from random import shuffle
from random import choice

class PokerCards:
        
    def __init__(self, case_color = 'blue'):
        self.case_color = case_color
        self.deck = []
        for i in ["heart", "spade", "club", "diamond"]:
            for j in range(1, 14):
                self.deck.append(Card(i, j))
        self.deck.append(Card("black joker", 14))
        self.deck.append(Card("red joker", 15))
                                        
    def shuffle_deck(self):
        shuffle(self.deck)
        print("Deck shuffled.")
        
    def random_toss(self, num = 1):
        print(f"{num} cards tossed:")
        for i in range(0, num):
            random_card = choice(self.deck)
            while random_card.rank == 3 and random_card.suit == "heart":
                random_card = choice(self.deck)
            self.deck.remove(random_card)
            print(f"  {random_card.rank} of {random_card.suit}")
    
    def __len__(self):
        return len(self.deck)

In [31]:
from random import shuffle

class Table:
    
    def __init__(self, n = 0):
        
        print("Setting Table...")
        self.players = []
        self.n = int(input("How many players do you want in this game? "))
        while self.n not in range(3,7):
            self.n = int(input("Wrong Input! Please choose between 3 and 6."))
        self.pk = PokerCards()
        
    def generate_players(self):
        
        for i in range(0,self.n):
            self.players.append(Player())
            
    def empty_hands(self):
        
        for i in self.players:
            i.hand = []
        print("All hands empty.")
            
    def deal_cards(self):
        
        self.pk.random_toss(54 % self.n)
        self.pk.shuffle_deck()
        i = 0
        while i < self.n:
            j = i
            while (j < len(self.pk)):
                self.players[i].hand.append(self.pk.deck[j])
                j += self.n
            i += 1
        print("All cards drawn.")
    
    def show_hands(self):
        
        for i in range(0, self.n):
            print(f"Player{i} has:")
            for j in self.players[i].hand:
                print(f"  {j.rank} of {j.suit}")
                    
    def sort_hands(self):
        
        for i in self.players:
            i.hand.sort(key = lambda x : x.rank)
        print("All hands sorted.")
    
    def first_player(self):
        
        for i in range(0, self.n):
            if self.players[i].has_3ofheart():
                return [self.players[i], i]
    
    '''
    def has_win(self):
        
        for i in range(0, self.n):
            if self.players[i].hand == []:
                return [self.player[i], i]
        return False  
    '''

    def check_deficit(self):
    
        for i in range(0, self.n):
            if self.players[i].wallet < 100:
                print(f"Player{i}'s Wallet is Empty!\nGame Over!\n")
                return True
        return False

In [32]:
class Player(Person):
    
    def __init__(self):
        self.wallet = randint(100, 1000)
        self.hand = []

    '''
    def draw(self, card):
        self.hand.append(card)
        print(f"Drew: {card.rank} of {card.suit}")    
                
    def toss(self, card):
        self.hand.remove(card)
        print(f"Tossed: {card.rank} of {card.suit}")
    '''       
        
    def has_3ofheart(self):
        for i in self.hand:
            if i.rank == 3 and i.suit == "heart":
                return True
        return False
        
    def beat(self, card):
        for i in range(0, len(self.hand)):
            if self.hand[i].rank > card.rank:
                p = self.hand.pop(i)                
                if self.hand == []:
                    return "Empty"
                return p
        return "No Beat"

In [33]:
class Game:
    
    def __init__(self):
        
        self.tb = Table()
        self.tb.generate_players()
            
    def checkout(self, winner_index):
    
        for i in range(0, self.tb.n):
            if i == winner_index:
                self.tb.players[i].wallet += (self.tb.n - 1) * 100
                print(f"Winner Player{i}'s wallet is now {self.tb.players[i].wallet}")
                continue
            self.tb.players[i].wallet -= 100
            print(f"Player{i}'s wallet is now {self.tb.players[i].wallet}")
        print("Checkout Completed.\n")
        
    def play(self):
        while not self.tb.check_deficit():
            self.tb.deal_cards()
            self.tb.sort_hands()
            self.tb.show_hands()
            print("\n****** Game Started! ******\n")
            first_circle = 1
            fi = self.tb.first_player()[1]
            print(f"First goes Player{fi}")
            fc = self.tb.first_player()[0].hand.pop(0)
            print(f"Player{fi}:\n  tossed {fc.rank} of {fc.suit}")
            cur = [fc, fi]
            has_win = 0
            while not has_win:
                
                i = 0
                while i < self.tb.n:
                    
                    if first_circle == 1:
                        first_circle = 0
                        if fi == self.tb.n - 1:
                            break
                        elif fi < self.tb.n - 1:
                            i = fi + 1
                    
                    if cur[1] == i:
                        p = self.tb.players[i].hand.pop(0)
                        print(f"Player{i} (again):\n  tossed {p.rank} of {p.suit}")
                        if self.tb.players[i].hand == []:
                            has_win = 1
                            print(f"\n****** Player{i} Won! ******\n")
                            break
                        cur[0] = p
                        if i < self.tb.n - 1:
                            i += 1
                        else:
                            break
                        
                    bt = self.tb.players[i].beat(cur[0])                    
                    if bt == "Empty":
                        has_win = 1
                        print(f"Player{i}:\n  tossed the last card.")
                        print(f"\n****** Player{i} Won! ******\n")
                        break
                    elif bt == "No Beat":
                        print(f"Player{i}:\n  passed")
                        i += 1
                        continue
                    else:
                        print(f"Player{i}:\n  tossed {bt.rank} of {bt.suit}")
                        cur = [bt, i]
                    i += 1
                                    
            self.tb.empty_hands()
            self.checkout(i)

In [34]:
Game().play()

Setting Table...
How many players do you want in this game? 6
0 cards tossed:
Deck shuffled.
All cards drawn.
All hands sorted.
Player0 has:
  1 of spade
  2 of spade
  3 of diamond
  3 of club
  7 of heart
  8 of heart
  12 of diamond
  13 of spade
  15 of red joker
Player1 has:
  1 of club
  2 of heart
  4 of diamond
  5 of spade
  5 of club
  6 of diamond
  7 of diamond
  9 of diamond
  13 of diamond
Player2 has:
  4 of club
  5 of diamond
  6 of heart
  6 of spade
  9 of club
  9 of spade
  10 of spade
  11 of heart
  11 of spade
Player3 has:
  1 of heart
  5 of heart
  6 of club
  8 of spade
  10 of diamond
  11 of diamond
  11 of club
  12 of club
  12 of spade
Player4 has:
  2 of diamond
  2 of club
  3 of spade
  3 of heart
  4 of spade
  7 of spade
  9 of heart
  10 of heart
  12 of heart
Player5 has:
  1 of diamond
  4 of heart
  7 of club
  8 of club
  8 of diamond
  10 of club
  13 of heart
  13 of club
  14 of black joker

****** Game Started! ******

First goes Player4
Pl